In [ ]:
# code sketch:
# 1. small subset of songs, get lyrics / info from spotify / genius api
#     turn lyrics coupled w metadata into vector using BERT or something
#.    store in Pinecone? FAISS
# 2. user input > gpt 4 / langchain to get semantic vector rep. of that
#.     compare using cosine sim., closest 10 get suggested


In [1]:
# import all necessary files
from setup import *
from spotify_handler import *
from genius_handler import *
#from langchain_handler import *

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
songNumber = 5 # number of songs we're gonna sample from randomly
pbar = tqdm(total = songNumber) #progress bar init
query_offset = 600
max_workers = 8

trackList = build_track_list(songNumber, query_offset)


  0%|          | 0/5 [00:00<?, ?it/s]

Error: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))


Daft Punk Infinity Repeating (2013 Demo) [feat. Julian Casablancas+The Voidz]


No lyrics found for Do Ya Like X Resonance - Sped Up by noturgf.
noturgf Do Ya Like X Resonance - Sped Up


Zonara Forms


No lyrics found for Relaxing Brown Noise 256 Hz Smooth Q by Jane Color.
Jane Color Relaxing Brown Noise 256 Hz Smooth Q


100%|██████████| 5/5 [00:41<00:00,  8.26s/it]

jagger finn Vas


In [4]:
print(len(trackList))

5


In [ ]:
#LANGCHAIN TIME
# here gonna take each lyric file and summarize it / vectorize it and store in pinecone
# initialize langchain / openAI

docs=[]
summary = []

for filename in tqdm(os.listdir(directory)):
    if filename.endswith(".txt"):  # check if the file is a .txt file
        file_path = os.path.join(directory, filename)
        # now you can do something with the file
        loader = TextLoader(file_path)
        doc = loader.load()
        docs.append(doc)
for i in tqdm(docs):
    summary.append(chain.run(i))

        

In [ ]:
pinecone.init(
    api_key=PINECONE_API_KEY,
    environment='asia-northeast1-gcp'
)

index_name="spotifysemantic"

for i in tqdm(docs):
    docsearch=Pinecone.from_documents(i, embeddings, index_name=index_name)





In [ ]:
docsearch = Pinecone.from_existing_index(index_name, embeddings)
query = "songs for the best day of my life"
ans = docsearch.similarity_search(query)

for i in ans:
    print(i.metadata)